### Pandas Lab: Time Shifts & Multi Level Indexing

This lab is designed to introduce you to working with time in a more granular way, and understanding how to build features when your data has hierarchies or panels.  

Ie, when you have repeated observations for the same objects.  This is an important concept because lots of statistical methods don't explicitly account for values which might naturally be correlated with one another over time.  

But lots of data **is** highly correlated over time!  

By the time you're done with this lab, you'll have built 9 columns that capture a variety of information about how an observed value is changing with respect to itself.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('/Users/cameronlefevre/Test-Repo/ClassMaterial/Unit2/data/restaurants.csv', parse_dates=['visit_date'])

**Question 1:** To capture some other aspects of dates, create columns in your dataset that capture the following aspects of each timestamp:

  - What quarter it's in
  - What month it's in
  - What year it's in
  - The numeric value of the `visit_date` column (ie, turn it into an integer)

If you want to try adding different pandas date parts, you can find them here:  https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#time-date-components

In [3]:
df.head()

,id,visit_date,visitors,calendar_date,day_of_week,holiday,genre,area,latitude,longitude,reserve_visitors
0,air_ba937bf13d40fb24,2016-01-13,25,2016-01-13,Wednesday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN
1,air_ba937bf13d40fb24,2016-01-14,32,2016-01-14,Thursday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN
2,air_ba937bf13d40fb24,2016-01-15,29,2016-01-15,Friday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN
3,air_ba937bf13d40fb24,2016-01-16,22,2016-01-16,Saturday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN
4,air_ba937bf13d40fb24,2016-01-18,6,2016-01-18,Monday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN


In [9]:
df['quarter'] = df['visit_date'].dt.quarter
df['month'] = df['visit_date'].dt.month
df['year'] = df['visit_date'].dt.year
df['day'] = df['visit_date'].astype(np.int64)

In [10]:
df.head()

,id,visit_date,visitors,calendar_date,day_of_week,holiday,genre,area,latitude,longitude,reserve_visitors,quarter,month,year,day
0,air_ba937bf13d40fb24,2016-01-13,25,2016-01-13,Wednesday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,1,1,2016,1452643200000000000
1,air_ba937bf13d40fb24,2016-01-14,32,2016-01-14,Thursday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,1,1,2016,1452729600000000000
2,air_ba937bf13d40fb24,2016-01-15,29,2016-01-15,Friday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,1,1,2016,1452816000000000000
3,air_ba937bf13d40fb24,2016-01-16,22,2016-01-16,Saturday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,1,1,2016,1452902400000000000
4,air_ba937bf13d40fb24,2016-01-18,6,2016-01-18,Monday,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,1,1,2016,1453075200000000000


**Question 2:** Set the multi-level index so the first level is the store id, and the second level is the date.  Make sure the date column is sorted in ascending order.  You might have to use the `sort_index(level=0)` method to get the values straight.

In [14]:
df = df.set_index(['id', 'visit_date']).sort_index(level=0)

**Question 3:** Time Series Embedding

Lots of times if you're trying to predict the value of something tomorrow, the most import piece of information is what the value of something is today, and yesterday, and so on.

However, your data won't really "know" about those values unless they can be observed alongside the current observation.  Data is read in as rows, not columns.  

To that end, make three columns that capture the value of the following:

 - What the previous recorded attendance for each restaurant was
 - The attendance from two days ago
 - The attendance from 7 days ago (ie, week over week)
 
**Remember:** This has to be done on a particular level of the index so make sure it's getting applied appropriately!

In [23]:
df['visitors_1dayprior'] = df.groupby(level=0)['visitors'].shift()
df['visitors_2dayprior'] = df.groupby(level=0)['visitors'].shift(2)
df['visitors_7dayprior'] = df.groupby(level=0)['visitors'].shift(7)

In [24]:
df.head()

visitors calendar_date day_of_week  holiday  \
id                   visit_date                                                
air_00a91d42b08b08d9 2016-07-01        35    2016-07-01      Friday        0   
                     2016-07-02         9    2016-07-02    Saturday        0   
                     2016-07-04        20    2016-07-04      Monday        0   
                     2016-07-05        25    2016-07-05     Tuesday        0   
                     2016-07-06        29    2016-07-06   Wednesday        0   

                                          genre  \
id                   visit_date                   
air_00a91d42b08b08d9 2016-07-01  Italian/French   
                     2016-07-02  Italian/French   
                     2016-07-04  Italian/French   
                     2016-07-05  Italian/French   
                     2016-07-06  Italian/French   

                                                            area   latitude  \
id                   visit_date                                               
air_00a91d42b08b08d9 2016-07-01  Tōkyō-to Chiyoda-ku Kudanminami  35.694003   
                     2016-07-02  Tōkyō-to Chiyoda-ku Kudanminami  35.694003   
                     2016-07-04  Tōkyō-to Chiyoda-ku Kudanminami  35.694003   
                     2016-07-05  Tōkyō-to Chiyoda-ku Kudanminami  35.694003   
                     2016-07-06  Tōkyō-to Chiyoda-ku Kudanminami  35.694003   

                                  longitude  reserve_visitors  quarter  month  \
id                   visit_date                                                 
air_00a91d42b08b08d9 2016-07-01  139.753595               NaN        3      7   
                     2016-07-02  139.753595               4.0        3      7   
                     2016-07-04  139.753595               NaN        3      7   
                     2016-07-05  139.753595               NaN        3      7   
                     2016-07-06  139.753595               NaN        3      7   

                                 year                  day  \
id                   visit_date                              
air_00a91d42b08b08d9 2016-07-01  2016  1467331200000000000   
                     2016-07-02  2016  1467417600000000000   
                     2016-07-04  2016  1467590400000000000   
                     2016-07-05  2016  1467676800000000000   
                     2016-07-06  2016  1467763200000000000   

                                 visitors_1dayprior  visitors_2dayprior  \
id                   visit_date                                           
air_00a91d42b08b08d9 2016-07-01                 NaN                 NaN   
                     2016-07-02                35.0                 NaN   
                     2016-07-04                 9.0                35.0   
                     2016-07-05                20.0                 9.0   
                     2016-07-06                25.0                20.0   

                                 visitors_7dayprior  
id                   visit_date                      
air_00a91d42b08b08d9 2016-07-01                 NaN  
                     2016-07-02                 NaN  
                     2016-07-04                 NaN  
                     2016-07-05                 NaN  
                     2016-07-06                 NaN

**Question 4:** Window Statistics

Lots of times, we want to capture some idea of momentum, or how some value changes with what's usually observed.

Ie, if we had 48 purchases in a store today, how does that number compare to what's happened in the last 14 days?  Are things trending up or trending down?  

This also allows us to get a clearer picture of general trends in values, even if there are irregular daily spikes.

To handle these sorts of issues, pandas has an entire section to calculate window statistics called `rolling`, it works like this:

In [9]:
# I'll create a sample dataframe with 30 days worth of values
import numpy as np
index = pd.date_range(start='01/01/2020', end='02/05/2020')
sample_df = pd.DataFrame(np.random.randn(36), index=index, columns=['Value'])
# and here's what it looks like
sample_df.head()

,Value
2020-01-01,-0.253379
2020-01-02,-0.838158
2020-01-03,-1.131807
2020-01-04,-1.708901
2020-01-05,-0.196300


In [11]:
# and now we'll see rolling 10 day averages
sample_df.rolling(10).mean()

,Value
2020-01-01,NaN
2020-01-02,NaN
2020-01-03,NaN
2020-01-04,NaN
2020-01-05,NaN
2020-01-06,NaN
2020-01-07,NaN
2020-01-08,NaN
2020-01-09,NaN
2020-01-10,-0.366059


You can specify the number of observations to calculate, and choose your aggregator -- `mean()`, `min()`, `sum()`, etc, although `mean()` is the most common.

**Your Turn:** Calculate the rolling 7, 25, and 60 day moving averages for visits for each restaurant inside the dataset.

**Note:** Do *not* try and merge them back into your dataset yet, just make sure you have the values showing up and save them as variables

And be mindful of performing these on the appropriate levels of your dataset.

In [27]:
df.groupby(level=0)['visitors'].rolling(7).mean()
df.groupby(level=0)['visitors'].rolling(25).mean()
df.groupby(level=0)['visitors'].rolling(60).mean()

id                    id                    visit_date
air_00a91d42b08b08d9  air_00a91d42b08b08d9  2016-07-01         NaN
                                            2016-07-02         NaN
                                            2016-07-04         NaN
                                            2016-07-05         NaN
                                            2016-07-06         NaN
                                                            ...   
air_fff68b929994bfbd  air_fff68b929994bfbd  2017-04-18    5.000000
                                            2017-04-19    4.428571
                                            2017-04-20    4.571429
                                            2017-04-21    4.428571
                                            2017-04-22    4.142857
Name: visitors, Length: 252108, dtype: float64

If you take a look at the index, you should notice that it has *three* levels to it, and not just two like before.  

Combining datasets with differing numbers of levels is cumbersome, and there's a decent amount of churn in what methods work from one version of Pandas to another.  

For now, try and get these values back into your original dataset by just using the `values` attribute, which will strip away the index and just return the values from the calculations.
 
So as a quick example, it would sort of work like this:

`five_day = df.groupby(level=0)['Visits'].your_stuff_here.values`

Take the values from the your previous calculations, and use them to create new columns for each one.

In [28]:
df['7dayrolling'] = df.groupby(level=0)['visitors'].rolling(7).mean().values
df['25dayrolling'] = df.groupby(level=0)['visitors'].rolling(25).mean().values
df['60dayrolling'] = df.groupby(level=0)['visitors'].rolling(60).mean().values

In [29]:
df.head()

visitors calendar_date day_of_week  holiday  \
id                   visit_date                                                
air_00a91d42b08b08d9 2016-07-01        35    2016-07-01      Friday        0   
                     2016-07-02         9    2016-07-02    Saturday        0   
                     2016-07-04        20    2016-07-04      Monday        0   
                     2016-07-05        25    2016-07-05     Tuesday        0   
                     2016-07-06        29    2016-07-06   Wednesday        0   

                                          genre  \
id                   visit_date                   
air_00a91d42b08b08d9 2016-07-01  Italian/French   
                     2016-07-02  Italian/French   
                     2016-07-04  Italian/French   
                     2016-07-05  Italian/French   
                     2016-07-06  Italian/French   

                                                            area   latitude  \
id                   visit_date                                               
air_00a91d42b08b08d9 2016-07-01  Tōkyō-to Chiyoda-ku Kudanminami  35.694003   
                     2016-07-02  Tōkyō-to Chiyoda-ku Kudanminami  35.694003   
                     2016-07-04  Tōkyō-to Chiyoda-ku Kudanminami  35.694003   
                     2016-07-05  Tōkyō-to Chiyoda-ku Kudanminami  35.694003   
                     2016-07-06  Tōkyō-to Chiyoda-ku Kudanminami  35.694003   

                                  longitude  reserve_visitors  quarter  month  \
id                   visit_date                                                 
air_00a91d42b08b08d9 2016-07-01  139.753595               NaN        3      7   
                     2016-07-02  139.753595               4.0        3      7   
                     2016-07-04  139.753595               NaN        3      7   
                     2016-07-05  139.753595               NaN        3      7   
                     2016-07-06  139.753595               NaN        3      7   

                                 year                  day  \
id                   visit_date                              
air_00a91d42b08b08d9 2016-07-01  2016  1467331200000000000   
                     2016-07-02  2016  1467417600000000000   
                     2016-07-04  2016  1467590400000000000   
                     2016-07-05  2016  1467676800000000000   
                     2016-07-06  2016  1467763200000000000   

                                 visitors_1dayprior  visitors_2dayprior  \
id                   visit_date                                           
air_00a91d42b08b08d9 2016-07-01                 NaN                 NaN   
                     2016-07-02                35.0                 NaN   
                     2016-07-04                 9.0                35.0   
                     2016-07-05                20.0                 9.0   
                     2016-07-06                25.0                20.0   

                                 visitors_7dayprior  7dayrolling  \
id                   visit_date                                    
air_00a91d42b08b08d9 2016-07-01                 NaN          NaN   
                     2016-07-02                 NaN          NaN   
                     2016-07-04                 NaN          NaN   
                     2016-07-05                 NaN          NaN   
                     2016-07-06                 NaN          NaN   

                                 25dayrolling  60dayrolling  
id                   visit_date                              
air_00a91d42b08b08d9 2016-07-01           NaN           NaN  
                     2016-07-02           NaN           NaN  
                     2016-07-04           NaN           NaN  
                     2016-07-05           NaN           NaN  
                     2016-07-06           NaN           NaN